In [107]:
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import seaborn as sns
from matplotlib.pyplot import figure
import psycopg2
import plotly.express as px
import plotly.graph_objects as go

In [108]:
def connection(sql):
    con = psycopg2.connect(host='35.247.202.234', database='dw',
    user='consulta', password='teste123')
    cur = con.cursor()    
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close();
    return df

In [109]:
sql_dimensoes_alunos = 'SELECT * FROM dados_sap.dimensoes_alunos'
sql_dimensoes_turmas ='SELECT * FROM dados_sap.dimensoes_turma'
sql_fatores_escola ='SELECT * FROM dados_sap.fatores_escola'
sql_fatores_turma ='SELECT * FROM dados_sap.fatores_turma'
sql_fatores_alunos = 'SELECT * FROM dados_sap.fatores_alunos'
#df = connection(sql_fatores_escola)

In [110]:
data_aluno = pd.read_csv('/home/eltonss/Documents/MEC/data/aluno.csv');
data_turma = pd.read_csv('/home/eltonss/Documents/MEC/data/turma.csv');
data_escola = pd.read_csv('/home/eltonss/Documents/MEC/data/escola.csv');

In [111]:
data_aluno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076 entries, 0 to 14075
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_aluno       14076 non-null  int64  
 1   id_turma       14076 non-null  int64  
 2   cod_matricula  9 non-null      float64
 3   nome_turma     8 non-null      object 
 4   nome_aluno     14076 non-null  object 
 5   tp_sexo        8 non-null      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 659.9+ KB


In [112]:
data_turma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_turma      382 non-null    int64 
 1   id_escola     382 non-null    int64 
 2   nome_turma    382 non-null    object
 3   modo_ensino   360 non-null    object
 4   etapa_ensino  360 non-null    object
 5   ano_turma     382 non-null    object
 6   data_criacao  382 non-null    object
dtypes: int64(2), object(5)
memory usage: 21.0+ KB


In [113]:
cols_to_use = data_turma.columns.difference(data_aluno.columns)
#print(cols_to_use)
data_aluno_turma = pd.merge(data_aluno, data_turma[cols_to_use], left_index=True, right_index=True, how='outer')
#data_aluno_turma = pd.merge(data_aluno, data_turma, on='id_turma')

data_aluno_turma.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14076 entries, 0 to 14075
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_aluno       14076 non-null  int64  
 1   id_turma       14076 non-null  int64  
 2   cod_matricula  9 non-null      float64
 3   nome_turma     8 non-null      object 
 4   nome_aluno     14076 non-null  object 
 5   tp_sexo        8 non-null      object 
 6   ano_turma      382 non-null    object 
 7   data_criacao   382 non-null    object 
 8   etapa_ensino   360 non-null    object 
 9   id_escola      382 non-null    float64
 10  modo_ensino    360 non-null    object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.3+ MB


In [114]:
data_aluno_turma[data_aluno_turma['id_escola']==11000058]

,id_aluno,id_turma,cod_matricula,nome_turma,nome_aluno,tp_sexo,ano_turma,data_criacao,etapa_ensino,id_escola,modo_ensino
361,5095,1015,NaN,NaN,ANNY VITÓRIA DA SILVA NASCIMENTO,NaN,6º,2022-05-01,NaN,11000058.0,NaN
362,5096,1015,NaN,NaN,ANTONIO CLEUTON CARNEIRO DA SILVA,NaN,5º,2022-08-23,NaN,11000058.0,NaN


In [115]:
data_aluno_turma_group = data_aluno_turma.groupby(['id_escola']).agg({"id_aluno": ["count"]}, split_out=4)
data_aluno_turma_group = data_aluno_turma_group.reset_index()
data_aluno_turma_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (id_escola, )      34 non-null     float64
 1   (id_aluno, count)  34 non-null     int64  
dtypes: float64(1), int64(1)
memory usage: 672.0 bytes


In [116]:
data_escola.rename(columns={'cod_escola': 'id_escola',}, inplace=True)
data_escola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131804 entries, 0 to 131803
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   resticao_de_atendimento     131804 non-null  object
 1   escola                      131804 non-null  object
 2   id_escola                   131804 non-null  int64 
 3   uf                          131804 non-null  object
 4   municipio                   131804 non-null  object
 5   localizacao                 131804 non-null  object
 6   localidade_diferenciada     131804 non-null  object
 7   categoria_administrativa    131804 non-null  object
 8   endereco                    131804 non-null  object
 9   telefone                    112040 non-null  object
 10  dependencia_administrativa  131804 non-null  object
 11  categoria_escola            131804 non-null  object
 12  conveniada_poder_publico    131804 non-null  object
 13  regulamentacao_conselho     1

In [118]:
cols_to_use = data_escola.columns.difference(data_aluno_turma.columns)
#print(cols_to_use)
data_aluno_turma_escola = pd.merge(data_aluno_turma, data_escola[cols_to_use], 
                                   left_index=True, right_index=True, how='outer')
data_aluno_turma_escola.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131804 entries, 0 to 131803
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id_aluno                    14076 non-null   float64
 1   id_turma                    14076 non-null   float64
 2   cod_matricula               9 non-null       float64
 3   nome_turma                  8 non-null       object 
 4   nome_aluno                  14076 non-null   object 
 5   tp_sexo                     8 non-null       object 
 6   ano_turma                   382 non-null     object 
 7   data_criacao                382 non-null     object 
 8   etapa_ensino                360 non-null     object 
 9   id_escola                   382 non-null     float64
 10  modo_ensino                 360 non-null     object 
 11  categoria_administrativa    131804 non-null  object 
 12  categoria_escola            131804 non-null  object 
 13  cod_estado    

In [119]:
data_aluno_turma_escola[data_aluno_turma_escola['id_escola']== 12006688.0]

,id_aluno,id_turma,cod_matricula,nome_turma,nome_aluno,tp_sexo,ano_turma,data_criacao,etapa_ensino,id_escola,...,meso,micro,municipio,outras_ofertas,porte,regiao,regulamentacao_conselho,resticao_de_atendimento,telefone,uf
10,4744.0,1000.0,NaN,NaN,JOÃO LUCAS LIMA DA SILVA.,NaN,6º,2022-05-01,ENSINO FUNDAMENTAL DE 9 ANOS - 6º ANO,12006688.0,...,LESTE RONDONIENSE,ALVORADA D'OESTE,Porto Velho,NaN,Mais de 1000 matrículas de escolarização,Norte,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,(69) 32225808,RO
11,4745.0,1000.0,NaN,NaN,LUANA INÁCIO DUTRA.,NaN,6º,2022-05-01,ENSINO FUNDAMENTAL DE 9 ANOS - 6º ANO,12006688.0,...,LESTE RONDONIENSE,ALVORADA D'OESTE,Porto Velho,NaN,Entre 201 e 500 matrículas de escolarização,Norte,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,(69) 32272120,RO
12,4746.0,1000.0,NaN,NaN,LUARA INÁCIO DUTRA.,NaN,6º,2022-05-01,ENSINO FUNDAMENTAL DE 9 ANOS - 6º ANO,12006688.0,...,LESTE RONDONIENSE,ALVORADA D'OESTE,Porto Velho,NaN,Entre 501 e 1000 matrículas de escolarização,Norte,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,(69) 32223810,RO
13,4747.0,1000.0,NaN,NaN,LUÍDI DA SILVA ALVES,NaN,7º,2022-05-01,ENSINO FUNDAMENTAL DE 9 ANOS - 7º ANO,12006688.0,...,LESTE RONDONIENSE,ALVORADA D'OESTE,Porto Velho,Atendimento Educacional Especializado,Entre 201 e 500 matrículas de escolarização,Norte,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,(69) 32521161,RO
14,4748.0,1000.0,NaN,NaN,MAIKOM SOUZA BEZERRA,NaN,9º,2022-05-01,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,12006688.0,...,LESTE RONDONIENSE,ALVORADA D'OESTE,Porto Velho,Atendimento Educacional Especializado,Entre 501 e 1000 matrículas de escolarização,Norte,Em Tramitação,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,(69) 992537434,RO


In [120]:
data_alunos_dimensoes = pd.read_csv('/home/eltonss/Documents/MEC/data/dimensoes_alunos.csv');
data_alunos_dimensoes.info()
df = data_alunos_dimensoes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   aluno              310 non-null    object
 1   fatores_dimensões  310 non-null    object
 2   value              310 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 7.4+ KB


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   aluno              310 non-null    object
 1   fatores_dimensões  310 non-null    object
 2   value              310 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 7.4+ KB


In [122]:
set(df['fatores_dimensões'])

{'Estudante-Comunidade',
 'Estudante-Escola',
 'Estudante-Estudante',
 'Estudante-Familia',
 'Estudante-Profissionais da Escola'}

In [123]:
cross_tab_prop = df.pivot(index='aluno', columns='fatores_dimensões', values='value')

cross_tab_prop.reset_index(inplace=True)


In [124]:
cross_tab_prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   aluno                              62 non-null     object
 1   Estudante-Comunidade               62 non-null     int64 
 2   Estudante-Escola                   62 non-null     int64 
 3   Estudante-Estudante                62 non-null     int64 
 4   Estudante-Familia                  62 non-null     int64 
 5   Estudante-Profissionais da Escola  62 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 3.0+ KB


In [18]:
cross_tab_prop.rename(columns={'aluno': 'id_aluno'}, inplace=True)

In [125]:
cols_to_use = cross_tab_prop.columns.difference(data_aluno_turma_escola.columns)
#print(cols_to_use)
data_aluno_turma_escola_dimensoes = pd.merge(data_aluno_turma_escola, cross_tab_prop[cols_to_use], 
                                   left_index=True, right_index=True, how='outer')
data_aluno_turma_escola_dimensoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131804 entries, 0 to 131803
Data columns (total 38 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id_aluno                           14076 non-null   float64
 1   id_turma                           14076 non-null   float64
 2   cod_matricula                      9 non-null       float64
 3   nome_turma                         8 non-null       object 
 4   nome_aluno                         14076 non-null   object 
 5   tp_sexo                            8 non-null       object 
 6   ano_turma                          382 non-null     object 
 7   data_criacao                       382 non-null     object 
 8   etapa_ensino                       360 non-null     object 
 9   id_escola                          382 non-null     float64
 10  modo_ensino                        360 non-null     object 
 11  categoria_administrativa           1318

In [38]:
fig = px.parallel_coordinates(df_aux,color ='etapa_ensino',
                              dimensions=['Estudante-Comunidade', 'Estudante-Escola', 'Estudante-Estudante',
                                          'Estudante-Familia', 'Estudante-Profissionais da Escola'],
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              color_continuous_midpoint=2)
fig.show()

In [58]:
df_aux = data_aluno_turma_escola_dimensoes[~data_aluno_turma_escola_dimensoes['Estudante-Comunidade'].isna()]

lst = []
colunas =['Estudante-Escola',
          'Estudante-Profissionais da Escola',
           'Estudante-Familia',
          'Estudante-Comunidade',           
          'Estudante-Estudante',           
          ]





for nome in colunas:
    lst.append(df_aux[nome].mean())
print(lst)
fig = go.Figure(data=[go.Scatter(
    x=colunas,
    y=lst,
    mode='markers',
    marker=dict(
            size=[E_ESC_baixo,E_PROF_baixo,E_FAM_baixo,E_COM_baixo,E_EST_baixo],
            cmax=7,
            cmin=0,
            color=lst,
            colorbar=dict(
                title='Fator'
            ),
            colorscale='RdYlGn'
        )
)])

fig.show()

[1.435483870967742, 1.435483870967742, 1.435483870967742, 1.2903225806451613, 2.0161290322580645]


In [130]:
data_aluno_turma_escola_dimensoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131804 entries, 0 to 131803
Data columns (total 38 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id_aluno                           14076 non-null   float64
 1   id_turma                           14076 non-null   float64
 2   cod_matricula                      9 non-null       float64
 3   nome_turma                         8 non-null       object 
 4   nome_aluno                         14076 non-null   object 
 5   tp_sexo                            8 non-null       object 
 6   ano_turma                          382 non-null     object 
 7   data_criacao                       382 non-null     object 
 8   etapa_ensino                       360 non-null     object 
 9   id_escola                          382 non-null     float64
 10  modo_ensino                        360 non-null     object 
 11  categoria_administrativa           1318

In [132]:
df_aux = data_aluno_turma_escola_dimensoes.copy()

E_ESC_baixo = len(df_aux.loc[df_aux['Estudante-Escola'] <= 2.83])
E_ESC_medio = len(df_aux.loc[(df_aux['Estudante-Escola'] > 2.83) & (df_aux['Estudante-Escola'] <= 4)])
E_ESC_alto = len(df_aux.loc[df_aux['Estudante-Escola'] > 4])
E_ESC_total = df_dimensao.loc[df_dimensao['dimensao']=='Estudante-Escola',['quantidade']].sum()[0]

E_PROF_baixo = len(df_aux.loc[df_aux['Estudante-Profissionais da Escola'] <= 2.66])
E_PROF_medio = len(df_aux.loc[(df_aux['Estudante-Profissionais da Escola'] > 2.66) & (df_aux['Estudante-Profissionais da Escola'] <= 3.66)])
E_PROF_alto = len(df_aux.loc[df_aux['Estudante-Profissionais da Escola'] > 3.66])
E_PROF_total = df_dimensao.loc[df_dimensao['dimensao']=='Estudante-Profissionais da Escola',['quantidade']].sum()[0]


E_FAM_baixo = len(df_aux.loc[df_aux['Estudante-Familia'] <= 3.33])
E_FAM_medio = len(df_aux.loc[(df_aux['Estudante-Familia'] > 3.33) & (df_aux['Estudante-Familia'] <= 4.33)])
E_FAM_alto = len(df_aux.loc[df_aux['Estudante-Familia'] > 4.33])
E_FAM_total = df_dimensao.loc[df_dimensao['dimensao']=='Estudante-Familia',['quantidade']].sum()[0]


E_COM_baixo = len(df_aux.loc[df_aux['Estudante-Comunidade'] <= 2.44])
E_COM_medio = len(df_aux.loc[(df_aux['Estudante-Comunidade'] > 2.44) & (df_aux['Estudante-Comunidade'] <= 3.55)])
E_COM_alto = len(df_aux.loc[df_aux['Estudante-Comunidade'] > 3.55])
E_COM_total = df_dimensao.loc[df_dimensao['dimensao']=='Estudante-Comunidade',['quantidade']].sum()[0]


E_EST_baixo = len(df_aux.loc[df_aux['Estudante-Estudante'] <= 2.44])
E_EST_medio = len(df_aux.loc[(df_aux['Estudante-Estudante'] > 2.44) & (df_aux['Estudante-Estudante'] <= 3.41)])
E_EST_alto = len(df_aux.loc[df_aux['Estudante-Estudante'] > 3.41])
E_EST_total = df_dimensao.loc[df_dimensao['dimensao']=='Estudante-Estudante',['quantidade']].sum()[0]

data =[['Estudante-Escola','baixo',E_ESC_baixo,E_ESC_baixo/E_ESC_total],
       ['Estudante-Escola','medio',E_ESC_medio,E_ESC_medio/E_ESC_total],
       ['Estudante-Escola','alto',E_ESC_alto,E_ESC_alto/E_ESC_total],
       
       ['Estudante-Profissionais da Escola','baixo',E_PROF_baixo,E_PROF_baixo/E_PROF_total],
       ['Estudante-Profissionais da Escola','medio',E_PROF_medio,E_PROF_medio/E_PROF_total],
       ['Estudante-Profissionais da Escola','alto',E_PROF_alto,E_PROF_alto/E_PROF_total],
       
       ['Estudante-Familia','baixo',E_FAM_baixo,E_FAM_baixo/E_FAM_total],
       ['Estudante-Familia','medio',E_FAM_medio,E_FAM_medio/E_FAM_total],
       ['Estudante-Familia','alto',E_FAM_alto,E_FAM_alto/E_FAM_total],
       
       ['Estudante-Comunidade','baixo',E_COM_baixo,E_COM_baixo/E_COM_total],
       ['Estudante-Comunidade','medio',E_COM_medio,E_COM_medio/E_COM_total],
       ['Estudante-Comunidade','alto',E_COM_alto,E_COM_alto/E_COM_total],
       
       ['Estudante-Estudante','baixo',E_EST_baixo,E_EST_baixo/E_EST_total],
       ['Estudante-Estudante','medio',E_EST_medio,E_EST_medio/E_EST_total],
       ['Estudante-Estudante','alto',E_EST_alto,E_EST_alto/E_EST_total],
       
      ]
df_dimensao = pd.DataFrame(data, columns=['dimensao','nivel', 'quantidade','percentual'])
df_dimensao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dimensao    15 non-null     object 
 1   nivel       15 non-null     object 
 2   quantidade  15 non-null     int64  
 3   percentual  15 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 608.0+ bytes


In [127]:
fig = px.bar(df_dimensao, x="dimensao", y="percentual",color="nivel",
                 color_discrete_map={'alto': 'green','medio': 'yellow', 'baixo': 'red'}, 
                 labels={'quantidade':'<b>Quantidade</b>','dimensao':'<b>Dimensao</b>',},
                 title="<b>Dimensoes por Quantidade de Alunos</b>")

fig.update_layout(
        legend_title="<b>Nível</b>",title_x=0.5

    )

fig.update_traces(showlegend=True)
fig.show()

In [128]:
df_dimensao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dimensao    15 non-null     object 
 1   nivel       15 non-null     object 
 2   quantidade  15 non-null     int64  
 3   percentual  15 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 608.0+ bytes


In [129]:
fig = px.bar(df_dimensao, x="dimensao", y="quantidade",color="nivel",
                 color_discrete_map={'alto': 'green','medio': 'yellow', 'baixo': 'red'}, 
                 labels={'quantidade':'<b>Quantidade</b>','dimensao':'<b>Dimensao</b>',},
                 title="<b>Dimensoes por Quantidade de Alunos</b>")

fig.update_layout(
        legend_title="<b>Nível</b>",title_x=0.5

    )

fig.update_traces(showlegend=True)
fig.show()

In [52]:
df = connection(sql_fatores_turma)

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   aluno              132 non-null    object 
 1   fatores_dimensões  132 non-null    object 
 2   value              132 non-null    float64
dtypes: float64(1), object(2)
memory usage: 3.2+ KB


In [101]:
set(df['fatores_dimensões'])

{'Acessibilidade e frequência escolar',
 'Aspectos Emocionais e Afetivos',
 'Aspectos infracionais e violência',
 'Condições Materiais da Escola',
 'Condições Materiais do Estudante',
 'Distanciamento escola – comunidade',
 'Gravidez/parentalidade e atividades de cuidado',
 'Inflexibilidade Pedagógica',
 'Qualidade Pedagógica',
 'Reprovações/distorção idade –série',
 'Significados da Escolarização/ Engajamento',
 'Suporte Familiar'}

In [102]:
set(df['aluno'])

{'6 C',
 '6* ANO',
 '6º ANO B',
 '7° ANO A',
 '7° ANO B',
 '7º ANO A',
 '8° ANO B',
 '8º ANO A',
 '8º ANO B',
 '9º ANO',
 '9º ANO A'}

In [100]:
cross_tab_prop = df.pivot(index='aluno', columns='fatores_dimensões', values='value')

cross_tab_prop.reset_index(inplace=True)
cross_tab_prop

fatores_dimensões,aluno,Acessibilidade e frequência escolar,Aspectos Emocionais e Afetivos,Aspectos infracionais e violência,Condições Materiais da Escola,Condições Materiais do Estudante,Distanciamento escola – comunidade,Gravidez/parentalidade e atividades de cuidado,Inflexibilidade Pedagógica,Qualidade Pedagógica,Reprovações/distorção idade –série,Significados da Escolarização/ Engajamento,Suporte Familiar
0,6 C,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,6* ANO,2.00,2.00,3.00,1.00,3.00,2.00,2.00,2.00,1.00,3.00,1.00,3.00
2,6º ANO B,1.50,1.50,1.00,1.00,1.50,1.00,1.00,1.00,1.00,1.50,2.00,1.00
3,7° ANO A,1.29,1.57,1.00,1.00,1.43,1.14,1.00,1.14,1.43,1.43,1.86,1.71
4,7° ANO B,1.64,1.36,0.91,1.45,1.91,1.64,1.36,1.64,1.82,1.82,2.00,1.45
5,7º ANO A,1.00,1.00,1.33,1.00,2.00,1.00,1.00,1.00,1.00,2.00,1.67,1.00
6,8° ANO B,1.60,1.60,1.00,1.00,2.60,1.60,1.40,1.40,1.00,1.80,2.20,1.60
7,8º ANO A,1.83,1.78,0.94,1.28,2.06,1.67,1.39,1.72,1.72,1.89,2.22,1.78
8,8º ANO B,1.00,1.75,1.00,1.50,2.25,2.00,1.75,1.50,1.25,2.75,2.25,1.25
9,9º ANO,1.00,1.00,1.00,1.00,1.60,1.60,1.40,1.00,1.00,1.60,2.00,1.00


In [136]:
data_alunos_fatores = pd.read_csv('/home/eltonss/Documents/MEC/data/fatores_alunos.csv');
data_alunos_fatores.info()
df = data_alunos_fatores
set(df['fatores_dimensões'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   aluno              744 non-null    object
 1   fatores_dimensões  744 non-null    object
 2   value              744 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 17.6+ KB


{'Acessibilidade e frequência escolar',
 'Aspectos Emocionais e Afetivos',
 'Aspectos infracionais e violência',
 'Condições Materiais da Escola',
 'Condições Materiais do Estudante',
 'Distanciamento escola – comunidade',
 'Gravidez/parentalidade e atividades de cuidado',
 'Inflexibilidade Pedagógica',
 'Qualidade Pedagógica',
 'Reprovações/distorção idade –série',
 'Significados da Escolarização/ Engajamento',
 'Suporte Familiar'}

In [137]:
cross_tab_prop = df.pivot(index='aluno', columns='fatores_dimensões', values='value')

cross_tab_prop.reset_index(inplace=True)
cross_tab_prop.rename(columns={'aluno': 'id_aluno'}, inplace=True)
cross_tab_prop

fatores_dimensões,id_aluno,Acessibilidade e frequência escolar,Aspectos Emocionais e Afetivos,Aspectos infracionais e violência,Condições Materiais da Escola,Condições Materiais do Estudante,Distanciamento escola – comunidade,Gravidez/parentalidade e atividades de cuidado,Inflexibilidade Pedagógica,Qualidade Pedagógica,Reprovações/distorção idade –série,Significados da Escolarização/ Engajamento,Suporte Familiar
0,A14799,0,0,0,0,0,0,0,0,0,0,0,0
1,A17927,1,2,1,1,1,1,3,1,1,2,2,1
2,A17936,1,2,1,2,2,2,1,2,1,3,2,1
3,A17947,1,2,1,2,3,2,1,2,2,3,3,2
4,A18171,1,1,2,1,2,1,1,1,1,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,A19209,1,1,1,1,2,1,1,1,1,1,2,1
58,A19210,1,1,1,1,3,1,1,1,1,3,2,1
59,A19211,2,1,1,1,2,1,1,1,1,1,2,2
60,A19212,3,2,1,2,2,1,1,2,2,1,3,2


In [139]:
cols_to_use = cross_tab_prop.columns.difference(data_aluno_turma_escola.columns)
#print(cols_to_use)
data_aluno_turma_escola_fatores = pd.merge(data_aluno_turma_escola, cross_tab_prop[cols_to_use], 
                                   left_index=True, right_index=True, how='outer')
data_aluno_turma_escola_fatores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131804 entries, 0 to 131803
Data columns (total 44 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   id_aluno                                        14076 non-null   float64
 1   id_turma                                        14076 non-null   float64
 2   cod_matricula                                   9 non-null       float64
 3   nome_turma                                      8 non-null       object 
 4   nome_aluno                                      14076 non-null   object 
 5   tp_sexo                                         8 non-null       object 
 6   ano_turma                                       382 non-null     object 
 7   data_criacao                                    382 non-null     object 
 8   etapa_ensino                                    360 non-null     object 
 9   id_escola                 

In [154]:


df_aux = data_aluno_turma_escola_fatores.copy()

var = 'Condições Materiais da Escola';
limite_inferior = 3.65;
limite_superior = 5.33
E_ESC_cond_mat_escola_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_ESC_cond_mat_escola_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_ESC_cond_mat_escola_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_ESC_cond_mat_escola_total =E_ESC_cond_mat_escola_baixo+E_ESC_cond_mat_escola_medio+E_ESC_cond_mat_escola_alto
     
var = 'Condições Materiais do Estudante';
limite_inferior = 2;
limite_superior = 3.33
E_ESC_cond_mat_estudante_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_ESC_cond_mat_estudante_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_ESC_cond_mat_estudante_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_ESC_cond_mat_estudante_total = E_ESC_cond_mat_estudante_baixo+E_ESC_cond_mat_estudante_medio+E_ESC_cond_mat_estudante_alto

var = 'Inflexibilidade Pedagógica';
limite_inferior = 2.33;
limite_superior = 4.33
E_PROF_inf_ped_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_PROF_inf_ped_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_PROF_inf_ped_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_PROF_inf_ped_total = E_PROF_inf_ped_baixo+E_PROF_inf_ped_medio+E_PROF_inf_ped_alto

var = 'Qualidade Pedagógica';
limite_inferior = 2.33;
limite_superior = 4;
E_PROF_quali_ped_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_PROF_quali_ped_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_PROF_quali_ped_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_PROF_quali_ped_total = E_PROF_quali_ped_baixo+E_PROF_quali_ped_medio+E_PROF_quali_ped_alto


  
var = 'Suporte Familiar';
limite_inferior = 3;
limite_superior = 5;
E_FAM_suporte_fam_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_FAM_suporte_fam_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_FAM_suporte_fam_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_FAM_suporte_fam_total = E_FAM_suporte_fam_baixo+E_FAM_suporte_fam_medio+E_FAM_suporte_fam_alto


var = 'Gravidez/parentalidade e atividades de cuidado';
limite_inferior = 3.33;
limite_superior = 5;
E_FAM_gravidez_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_FAM_gravidez_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_FAM_gravidez_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_FAM_gravidez_total = E_FAM_gravidez_baixo+E_FAM_gravidez_medio+E_FAM_gravidez_alto
   

var = 'Aspectos infracionais e violência';
limite_inferior = 2;
limite_superior = 3.66;
E_COM_infracional_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_COM_infracional_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_COM_infracional_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_COM_infracional_total = E_COM_infracional_baixo+E_COM_infracional_medio+E_COM_infracional_alto



var = 'Distanciamento escola – comunidade';
limite_inferior = 2;
limite_superior = 3.66;
E_COM_dist_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_COM_dist_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_COM_dist_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_COM_dist_total = E_COM_dist_baixo+E_COM_dist_medio+E_COM_dist_alto


var = 'Acessibilidade e frequência escolar';
limite_inferior = 2;
limite_superior = 3.66;
E_COM_acessibilidade_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_COM_acessibilidade_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_COM_acessibilidade_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_COM_acessibilidade_total = E_COM_acessibilidade_baixo+E_COM_acessibilidade_medio+E_COM_acessibilidade_alto



var = 'Significados da Escolarização/ Engajamento';
limite_inferior = 2;
limite_superior = 4;
E_EST_engajamento_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_EST_engajamento_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_EST_engajamento_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_EST_engajamento_total = E_EST_engajamento_baixo+E_EST_engajamento_medio+E_EST_engajamento_alto



var = 'Aspectos Emocionais e Afetivos';
limite_inferior = 2.33;
limite_superior = 4;
E_EST_asp_emocional_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_EST_asp_emocional_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_EST_asp_emocional_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_EST_asp_emocional_total = E_EST_asp_emocional_baixo+E_EST_asp_emocional_medio+E_EST_asp_emocional_alto


var = 'Reprovações/distorção idade –série';
limite_inferior = 2;
limite_superior = 3.33;
E_EST_reprovacao_baixo = len(df_aux.loc[df_aux[var] <= limite_inferior])
E_EST_reprovacao_medio = len(df_aux.loc[(df_aux[var] > limite_inferior) & (df_aux[var] <= limite_superior)])
E_EST_reprovacao_alto = len(df_aux.loc[df_aux[var] > limite_superior])
E_EST_reprovacao_total = E_EST_reprovacao_baixo+E_EST_reprovacao_medio+E_EST_reprovacao_alto 



data =[
       ['Condições Materiais da Escola','baixo',E_ESC_cond_mat_escola_baixo,E_ESC_cond_mat_escola_baixo/E_ESC_cond_mat_escola_total],
       ['Condições Materiais da Escola','medio',E_ESC_cond_mat_escola_medio,E_ESC_cond_mat_escola_medio/E_ESC_cond_mat_escola_total],
       ['Condições Materiais da Escola','alto',E_ESC_cond_mat_escola_alto,E_ESC_cond_mat_escola_alto/E_ESC_cond_mat_escola_total],
       
       ['Condições Materiais do Estudante','baixo',E_ESC_cond_mat_estudante_baixo,E_ESC_cond_mat_estudante_baixo/E_ESC_cond_mat_estudante_total],
       ['Condições Materiais do Estudante','medio',E_ESC_cond_mat_estudante_medio,E_ESC_cond_mat_estudante_medio/E_ESC_cond_mat_estudante_total],
       ['Condições Materiais do Estudante','alto',E_ESC_cond_mat_estudante_alto,E_ESC_cond_mat_estudante_alto/E_ESC_cond_mat_estudante_total],
       
       ['Inflexibilidade Pedagógica','baixo',E_PROF_inf_ped_baixo,E_PROF_inf_ped_baixo/E_PROF_inf_ped_total],
       ['Inflexibilidade Pedagógica','medio',E_PROF_inf_ped_medio,E_PROF_inf_ped_medio/E_PROF_inf_ped_total],
       ['Inflexibilidade Pedagógica','alto',E_PROF_inf_ped_alto,E_PROF_inf_ped_alto/E_PROF_inf_ped_total],
    
       ['Qualidade Pedagógica','baixo',E_PROF_quali_ped_baixo,E_PROF_quali_ped_baixo/E_PROF_quali_ped_total],     
       ['Qualidade Pedagógica','medio',E_PROF_quali_ped_medio,E_PROF_quali_ped_medio/E_PROF_quali_ped_total],
       ['Qualidade Pedagógica','alto',E_PROF_quali_ped_alto,E_PROF_quali_ped_alto/E_PROF_quali_ped_total],

       ['Suporte Familiar','baixo',E_FAM_suporte_fam_baixo,E_FAM_suporte_fam_baixo/E_FAM_suporte_fam_total],     
       ['Suporte Familiar','medio',E_FAM_suporte_fam_medio,E_FAM_suporte_fam_medio/E_FAM_suporte_fam_total],
       ['Suporte Familiar','alto',E_FAM_suporte_fam_alto,E_FAM_suporte_fam_alto/E_FAM_suporte_fam_total],

       ['Gravidez/parentalidade e atividades de cuidado','baixo',E_FAM_gravidez_baixo,E_FAM_gravidez_baixo/E_FAM_gravidez_total],     
       ['Gravidez/parentalidade e atividades de cuidado','medio',E_FAM_gravidez_medio,E_FAM_gravidez_medio/E_FAM_gravidez_total],
       ['Gravidez/parentalidade e atividades de cuidado','alto',E_FAM_gravidez_alto,E_FAM_gravidez_alto/E_FAM_gravidez_total],
        
       ['Aspectos infracionais e violência','baixo',E_COM_infracional_baixo,E_COM_infracional_baixo/E_COM_infracional_total],     
       ['Aspectos infracionais e violência','medio',E_COM_infracional_medio,E_COM_infracional_medio/E_COM_infracional_total],
       ['Aspectos infracionais e violência','alto',E_COM_infracional_alto,E_COM_infracional_alto/E_COM_infracional_total],

       ['Distanciamento escola – comunidade','baixo',E_COM_dist_baixo,E_COM_dist_baixo/E_COM_dist_total],     
       ['Distanciamento escola – comunidade','medio',E_COM_dist_medio,E_COM_dist_medio/E_COM_dist_total],
       ['Distanciamento escola – comunidade','alto',E_COM_dist_alto,E_COM_dist_alto/E_COM_dist_total],

       ['Acessibilidade e frequência escolar','baixo',E_COM_acessibilidade_baixo,E_COM_acessibilidade_baixo/E_COM_acessibilidade_total],     
       ['Acessibilidade e frequência escolar','medio',E_COM_acessibilidade_medio,E_COM_acessibilidade_medio/E_COM_acessibilidade_total],
       ['Acessibilidade e frequência escolar','alto',E_COM_acessibilidade_alto,E_COM_acessibilidade_alto/E_COM_acessibilidade_total],
       
       ['Significados da Escolarização/ Engajamento','baixo',E_EST_engajamento_baixo,E_EST_engajamento_baixo/E_EST_engajamento_total],     
       ['Significados da Escolarização/ Engajamento','medio',E_EST_engajamento_medio,E_EST_engajamento_medio/E_EST_engajamento_total],
       ['Significados da Escolarização/ Engajamento','alto',E_EST_engajamento_alto,E_EST_engajamento_alto/E_EST_engajamento_total],

       ['Aspectos Emocionais e Afetivos','baixo',E_EST_asp_emocional_baixo,E_EST_asp_emocional_baixo/E_EST_asp_emocional_total],     
       ['Aspectos Emocionais e Afetivos','medio',E_EST_asp_emocional_medio,E_EST_asp_emocional_medio/E_EST_asp_emocional_total],
       ['Aspectos Emocionais e Afetivos','alto',E_EST_asp_emocional_alto,E_EST_asp_emocional_alto/E_EST_asp_emocional_total],

       ['Reprovações/distorção idade –série','baixo',E_EST_reprovacao_baixo,E_EST_reprovacao_baixo/E_EST_reprovacao_total],     
       ['Reprovações/distorção idade –série','medio',E_EST_reprovacao_medio,E_EST_reprovacao_medio/E_EST_reprovacao_total],
       ['Reprovações/distorção idade –série','alto',E_EST_reprovacao_alto,E_EST_reprovacao_alto/E_EST_reprovacao_total],

      ]
df_fator = pd.DataFrame(data, columns=['fator','nivel', 'quantidade','percentual'])
df_fator

,fator,nivel,quantidade,percentual
0,Condições Materiais da Escola,baixo,62,1.000000
1,Condições Materiais da Escola,medio,0,0.000000
2,Condições Materiais da Escola,alto,0,0.000000
3,Condições Materiais do Estudante,baixo,40,0.645161
4,Condições Materiais do Estudante,medio,22,0.354839
5,Condições Materiais do Estudante,alto,0,0.000000
6,Inflexibilidade Pedagógica,baixo,58,0.935484
7,Inflexibilidade Pedagógica,medio,4,0.064516
8,Inflexibilidade Pedagógica,alto,0,0.000000
9,Qualidade Pedagógica,baixo,59,0.951613


In [159]:
fig = px.bar(df_fator, x="fator", y="percentual",color="nivel",
                 color_discrete_map={'alto': 'green','medio': 'yellow', 'baixo': 'red'}, 
                 labels={'percentual':'<b>Percentual</b>','fator':'<b>Fator</b>',},
                 title="<b>Fatores por Quantidade de Alunos</b>")

fig.update_layout(
        legend_title="<b>Nível</b>",title_x=0.5

    )

fig.update_traces(showlegend=True)
fig.show()

In [158]:
fig = px.bar(df_fator, x="fator", y="quantidade",color="nivel",
                 color_discrete_map={'alto': 'green','medio': 'yellow', 'baixo': 'red'}, 
                 labels={'quantidade':'<b>Quantidade</b>','fator':'<b>Fator</b>',},
                 title="<b>Fatores por Quantidade de Alunos</b>")

fig.update_layout(
        legend_title="<b>Nível</b>",title_x=0.5

    )

fig.update_traces(showlegend=True)
fig.show()